# Aprenentatge Supervisat - Regressions

**Exercici 1**

Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

**Exercici 2**

Compara’ls en base al MSE i al R2 .

**Exercici 3**

Entrena’ls utilitzant els diferents paràmetres que admeten.

**Exercici 4**

Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna)

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

# Data

[Airlines Delay: Airline on-time statistics and delay causes](https://www.kaggle.com/giovamata/airlinedelaycauses) 

- Year: 1987-2008
- Month: 1-12
- DayofMonth: 1-31
- DayOfWeek: 1 (Monday) - 7 (Sunday)
- DepTime: departure time (local, hhmm)
- CRSDepTime: scheduled departure time (local, hhmm)
- ArrTime: arrival time (local, hhmm)
- CRSArrTime: scheduled arrival time (local, hhmm)
- UniqueCarrier: unique carrier code
- FlightNum: flight number
- TailNum: plane tail number 
- ActualElapsedTime: flygth time in minutes (Total)
- CRSElapsedTime: scheduled	flygth time in minutes (Total)
- AirTime: time on air in minutes
- ArrDelay:	arrival delay in minutes
- DepDelay: departure delay in minutes
- Origin: origin IATA airport code
- Dest:	destination IATA airport code
- Distance: distance in miles
- TaxiIn: taxi in time, in minutes (movement on ground)
- TaxiOut: taxi out time, in minutes (movement on ground)
- Cancelled: was the flight cancelled?
- CancellationCode:	[reason for cancellation](https://aspmhelp.faa.gov/index/Types_of_Delay.html) (A = carrier, B = weather, C = NAS, D = security)
- Diverted:	1 = yes, 0 = no ("Desviado")
- CarrierDelay:	delayed time due to Carrier in minutes
- WeatherDelay:	delayed time due to Weather in minutes
- NASDelay: delayed time due to NAS in minutes
- SecurityDelay: delayed time due to security in minuts
- LateAircraftDelay: delayed time due to late aircraft in minutes

In [2]:
# Read csv
df_raw = pd.read_csv("./archive/DelayedFlights.csv", index_col = [0])

/Users/luis/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
## Columns and Data types
df_raw.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1936758 entries, 0 to 7009727
Data columns (total 29 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               1936758 non-null  int64  
 1   Month              1936758 non-null  int64  
 2   DayofMonth         1936758 non-null  int64  
 3   DayOfWeek          1936758 non-null  int64  
 4   DepTime            1936758 non-null  float64
 5   CRSDepTime         1936758 non-null  int64  
 6   ArrTime            1929648 non-null  float64
 7   CRSArrTime         1936758 non-null  int64  
 8   UniqueCarrier      1936758 non-null  object 
 9   FlightNum          1936758 non-null  int64  
 10  TailNum            1936753 non-null  object 
 11  ActualElapsedTime  1928371 non-null  float64
 12  CRSElapsedTime     1936560 non-null  float64
 13  AirTime            1928371 non-null  float64
 14  ArrDelay           1928371 non-null  float64
 15  DepDelay           1936758 non-n

In [4]:
## Sample
df_raw.sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2756860,2008,5,23,5,1740.0,1730,2000.0,1953,FL,623,N922AT,140.0,143.0,126.0,7.0,10.0,MCO,PHL,861,5.0,9.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4931190,2008,9,29,1,850.0,820,1140.0,1125,WN,412,N660SW,110.0,125.0,93.0,15.0,30.0,MDW,PHL,668,5.0,12.0,0,N,0,1.0,0.0,0.0,0.0,14.0
5684572,2008,10,8,3,1332.0,1121,1544.0,1311,EV,4924,N686BR,72.0,50.0,40.0,153.0,131.0,MGM,ATL,147,16.0,16.0,0,N,0,131.0,0.0,22.0,0.0,0.0
6537952,2008,12,26,5,1509.0,1440,1618.0,1605,WN,670,N300SW,69.0,85.0,62.0,13.0,29.0,DTW,BWI,408,2.0,5.0,0,N,0,NaN,NaN,NaN,NaN,NaN
124071,2008,1,1,2,1844.0,1715,2039.0,1919,XE,3142,N17928,115.0,124.0,83.0,80.0,89.0,EWR,CMH,462,8.0,24.0,0,N,0,80.0,0.0,0.0,0.0,0.0
3202738,2008,6,24,2,1428.0,1345,1639.0,1550,OO,2511,N498CA,131.0,125.0,108.0,49.0,43.0,MCI,SAT,706,4.0,19.0,0,N,0,43.0,0.0,6.0,0.0,0.0
2092189,2008,4,19,6,1155.0,1142,1908.0,1918,DL,1056,N656DL,253.0,276.0,233.0,-10.0,13.0,SJC,ATL,2116,9.0,11.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4755909,2008,8,20,3,1506.0,1325,1625.0,1430,AA,1886,N082AA,79.0,65.0,39.0,115.0,101.0,MIA,MCO,193,23.0,17.0,0,N,0,0.0,75.0,14.0,0.0,26.0
4340682,2008,8,19,2,732.0,720,1002.0,934,XE,2566,N12126,210.0,194.0,186.0,28.0,12.0,IAH,SLC,1195,6.0,18.0,0,N,0,12.0,0.0,16.0,0.0,0.0
4982993,2008,9,9,2,1700.0,1635,1746.0,1715,OH,6660,N972CA,106.0,100.0,86.0,31.0,25.0,CVG,MCI,539,7.0,13.0,0,N,0,0.0,0.0,31.0,0.0,0.0


In [5]:
## Drop Duplicates
df_raw.drop_duplicates(inplace = True)

In [6]:
## Null Values %
df_raw.isnull().mean()*100

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               0.000000
CRSDepTime            0.000000
ArrTime               0.367109
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               0.000258
ActualElapsedTime     0.433044
CRSElapsedTime        0.010223
AirTime               0.433044
ArrDelay              0.433044
DepDelay              0.000000
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                0.367109
TaxiOut               0.023493
Cancelled             0.000000
CancellationCode      0.000000
Diverted              0.000000
CarrierDelay         35.588892
WeatherDelay         35.588892
NASDelay             35.588892
SecurityDelay        35.588892
LateAircraftDelay    35.588892
dtype: float64

In [7]:
## Columns with low percentage of nulls (less than 2% in total)
subset = ["ArrTime", "TailNum", "ActualElapsedTime", "CRSElapsedTime", 
          "AirTime", "ArrDelay", "TaxiIn", "TaxiOut"]
## Drop rows with low percentage of nulls
df_raw = df_raw.dropna(subset=subset)

In [8]:
## Transform DepTime and ArrTime to a more consistent notation (hh:mm)
df_raw["DepTime"] = df_raw["DepTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 
df_raw["CRSDepTime"] = df_raw["CRSDepTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 
df_raw["ArrTime"] = df_raw["ArrTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 
df_raw["CRSArrTime"] = df_raw["CRSArrTime"].astype(int).apply(lambda x: str(x).zfill(4)).apply(lambda x: x[0:2] + ":" + x[2:]) 

In [9]:
## Change dtypes
df_raw["FlightNum"] = df_raw["FlightNum"].astype(str)
df_raw["Cancelled"] = df_raw["Cancelled"].astype(str)
df_raw["Diverted"] = df_raw["Diverted"].astype(str)

In [10]:
## Divide into numerical and categorical
df_num = df_raw.select_dtypes(include = ["int64", "float64"])
df_cat = df_raw.select_dtypes(exclude = ["int64", "float64"])

In [11]:
## Describe num
df_num.describe().round(2)

,Year,Month,DayofMonth,DayOfWeek,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1928366.0,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1928366.00,1247484.00,1247484.00,1247484.00,1247484.00,1247484.00
mean,2008.0,6.11,15.75,3.98,133.31,134.20,108.28,42.20,43.09,764.95,6.81,18.22,19.18,3.70,15.02,0.09,25.30
std,0.0,3.48,8.78,2.00,72.06,71.23,68.64,56.78,53.27,573.89,5.27,14.31,43.55,21.49,33.83,2.02,42.05
min,2008.0,1.00,1.00,1.00,14.00,-21.00,0.00,-109.00,6.00,11.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2008.0,3.00,8.00,2.00,80.00,82.00,58.00,9.00,12.00,338.00,4.00,10.00,0.00,0.00,0.00,0.00,0.00
50%,2008.0,6.00,16.00,4.00,116.00,116.00,90.00,24.00,24.00,606.00,6.00,14.00,2.00,0.00,2.00,0.00,8.00
75%,2008.0,9.00,23.00,6.00,165.00,165.00,137.00,56.00,53.00,997.00,8.00,21.00,21.00,0.00,15.00,0.00,33.00
max,2008.0,12.00,31.00,7.00,1114.00,660.00,1091.00,2461.00,2467.00,4962.00,240.00,422.00,2436.00,1352.00,1357.00,392.00,1316.00


In [12]:
## Drop Year
df_raw.drop(columns = "Year", inplace = True)

In [13]:
## Describe cat
df_cat.describe()

,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,Origin,Dest,Cancelled,CancellationCode,Diverted
count,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366,1928366
unique,1438,1193,1440,1361,20,7498,5360,303,302,1,1,1
top,18:00,18:00,21:00,19:30,WN,16,N325SW,ATL,ORD,0,N,0
freq,3176,13867,2981,9148,376201,1575,961,131213,108265,1928366,1928366,1928366


In [14]:
## Drop Cancelled, CancellationCode and Diverted
df_raw.drop(columns = ["Cancelled", "CancellationCode", "Diverted"], inplace = True)

In [15]:
## Categorical column with delay > 15 min (1 = Yes, 0 = No)
df_raw["DelayCat"] = df_raw["ArrDelay"].apply(lambda x: 1 if x > 15 else 0)

In [16]:
## Mean Velocity columns in miles/min
df_raw = df_raw[df_raw["AirTime"] != 0]
df_raw["Velocity"] = df_raw["Distance"] / df_raw["AirTime"] 

In [17]:
## Origin-Destination Columns
df_raw["Fligth"] = df_raw["Origin"] + "-" + df_raw["Dest"]

In [18]:
## Save Final Dataframe
df_raw.to_csv("df_clean.csv")

# Exercisi 1

In [19]:
# Read csv
df_clean = pd.read_csv("df_clean.csv", index_col = [0])

/Users/luis/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
# Sample
df_clean.sample(10)

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DelayCat,Velocity,Fligth
5965448,11,10,1,12:16,12:10,12:39,12:40,WN,1726,N904WN,83.0,90.0,68.0,-1.0,6.0,PIT,MDW,402,9.0,6.0,NaN,NaN,NaN,NaN,NaN,0,5.911765,PIT-MDW
5693146,10,8,3,12:21,12:05,14:04,13:35,EV,5351,N872AS,103.0,90.0,83.0,29.0,16.0,OAJ,ATL,399,12.0,8.0,13.0,0.0,13.0,0.0,3.0,1,4.807229,OAJ-ATL
1179157,3,4,2,19:26,19:10,22:05,21:05,WN,2885,N714CB,159.0,115.0,104.0,60.0,16.0,IND,JAX,688,5.0,50.0,0.0,0.0,44.0,0.0,16.0,1,6.615385,IND-JAX
4380850,8,25,1,20:52,20:45,21:40,21:32,YV,1043,N27314,48.0,47.0,36.0,8.0,7.0,ITO,HNL,216,4.0,8.0,NaN,NaN,NaN,NaN,NaN,0,6.000000,ITO-HNL
2409705,5,9,5,21:45,21:20,22:33,22:15,WN,247,N644SW,48.0,55.0,34.0,18.0,25.0,MCO,FLL,178,4.0,10.0,18.0,0.0,0.0,0.0,0.0,1,5.235294,MCO-FLL
2737234,5,23,5,07:56,07:50,09:18,09:15,F9,420,N910FR,82.0,85.0,66.0,3.0,6.0,SLC,DEN,391,7.0,9.0,NaN,NaN,NaN,NaN,NaN,0,5.924242,SLC-DEN
1352451,3,5,3,22:05,21:45,23:55,23:25,OO,2642,N984CA,50.0,40.0,25.0,30.0,20.0,MKE,GRR,120,5.0,20.0,30.0,0.0,0.0,0.0,0.0,1,4.800000,MKE-GRR
764732,2,29,5,11:30,11:10,14:31,14:14,OH,5382,N969CA,121.0,124.0,104.0,17.0,20.0,OKC,CVG,756,2.0,15.0,17.0,0.0,0.0,0.0,0.0,1,7.269231,OKC-CVG
898474,2,21,4,19:02,18:35,21:51,21:24,DL,887,N936DL,169.0,169.0,149.0,27.0,27.0,BDL,ATL,859,11.0,9.0,0.0,0.0,0.0,0.0,27.0,1,5.765101,BDL-ATL
953819,2,22,5,18:52,17:59,20:54,19:28,FL,619,N288AT,122.0,89.0,96.0,86.0,53.0,TPA,ATL,406,15.0,11.0,0.0,0.0,33.0,0.0,53.0,1,4.229167,TPA-ATL


In [21]:
# Variables independientes
x = df_clean[["DepDelay", "Distance", "AirTime"]]
# Variables dependientes
y = df_clean["ArrDelay"]

In [22]:
# Regresión Lineal
model_1 = LinearRegression().fit(x,y)

# Arbol de regresión
model_2 = DecisionTreeRegressor().fit(x,y)

# Regresión Lasso
model_3 = LassoCV().fit(x,y)

# Exercisi 2

In [23]:
# Fitted values
y_1 = model_1.predict(x)
y_2 = model_2.predict(x)
y_3 = model_3.predict(x)

In [24]:
# MSE
RMSE_1 = mean_squared_error(y, y_1)
RMSE_2 = mean_squared_error(y, y_2)
RMSE_3 = mean_squared_error(y, y_3)

print(
    """
    Root MSE:
    Model 1: {}
    Model 2: {}
    Model 3: {}
    """.format(RMSE_1, RMSE_2, RMSE_3)
)


    Root MSE:
    Model 1: 245.7145687570543
    Model 2: 65.09371839944818
    Model 3: 245.77375226934674
    


In [25]:
# R2
R2_1 = r2_score(y, y_1)
R2_2 = r2_score(y, y_2)
R2_3 = r2_score(y, y_3)

print(
    """
    R2 score:
    Model 1: {}
    Model 2: {}
    Model 3: {}
    """.format(R2_1, R2_2, R2_3)
)


    R2 score:
    Model 1: 0.9237969223346831
    Model 2: 0.9798125861897027
    Model 3: 0.9237785678439155
    


El segon model consegueix explicar el 98% de la variablitat de la variable ArrDelay, mentre que els altre dos només el 92%. La suma dels errors al cuadrat es major per tant en el primer i el tercer model. 

# Exercici 3

In [26]:
# Regresión Lineal Normalizando
model_1 = LinearRegression(normalize=True).fit(x,y)

# Arbol de regresión Personalizado
model_2 = DecisionTreeRegressor(max_depth=20, max_leaf_nodes=10000, random_state=1).fit(x,y)

# Regresión Lasso
model_3 = LassoCV(normalize=True).fit(x,y)

In [27]:
# Fitted values
y_1 = model_1.predict(x)
y_2 = model_2.predict(x)
y_3 = model_3.predict(x)

In [28]:
# R2
R2_1 = r2_score(y, y_1)
R2_2 = r2_score(y, y_2)
R2_3 = r2_score(y, y_3)

print(
    """
    R2 score:
    Model 1: {}
    Model 2: {}
    Model 3: {}
    """.format(R2_1, R2_2, R2_3)
)


    R2 score:
    Model 1: 0.9237969223346831
    Model 2: 0.93147705998471
    Model 3: 0.92370083084097
    


- Al normalitzar les dades a la regresió lineal no en aquest cas no s'aconsegueix cap millora. 
- Cambiant alguns parametres del arbre de regresió, com pot ser l'alçada o la cantitat de fulles, varía la variabilitat explicada, en aquest cas, cap a pitjor. L'algoritme ja intentar optimitzar aquests factors. Nosaltres els podem alterar en situación específiques perque s'adaptin a les nostres necessitats.
- Al normalitzar les dades a la regresió LASSO tampoc notem cap millora. 

# Exercici 4

In [29]:
# Split into Test and Train
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33, random_state=1)

In [30]:
# Regresión Lineal Normalizando
model_1 = LinearRegression(normalize=True).fit(X_train,Y_train)

# Arbol de regresión Personalizado
model_2 = DecisionTreeRegressor(max_depth=20, max_leaf_nodes=10000, random_state=1).fit(X_train,Y_train)

# Regresión Lasso
model_3 = LassoCV(normalize=True).fit(X_train,Y_train)

In [31]:
# Predicted values
y_1 = model_1.predict(X_test)
y_2 = model_2.predict(X_test)
y_3 = model_3.predict(X_test)

In [32]:
# R2
R2_1 = r2_score(Y_test, y_1)
R2_2 = r2_score(Y_test, y_2)
R2_3 = r2_score(Y_test, y_3)

print(
    """
    R2 score:
    Model 1: {}
    Model 2: {}
    Model 3: {}
    """.format(R2_1, R2_2, R2_3)
)


    R2 score:
    Model 1: 0.9235715044294373
    Model 2: 0.9164089741913837
    Model 3: 0.9234956948925583
    


L'arbre de regresió funciona molt millor que els altres dos models alhora de predir obresvacions que s'han utilitzat al entrenament, degut a la manera en com ha estat construit. A l'hora de predir noves observacións, en aquest cas el conjunt de test, la seva capacitat explicativa cau. En canvi, els altres dos models aconsegueixen mantenirse bastant estables. 